In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [4]:
sess = tf.Session()

In [9]:
sess.run(x.initializer)
sess.run(y.initializer)

In [10]:
result = sess.run(f)

In [11]:
result

42

In [12]:
sess.close()

In [14]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result1 = f.eval()

In [15]:
result1

42

In [16]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [17]:
result

42

In [25]:
sess= tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


/Users/liufang/miniconda3/envs/gluon/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# Managing Graph

In [19]:
x1 = tf.Variable(1)

In [20]:
x1.graph is tf.get_default_graph()

True

In [21]:
graph = tf.Graph()

In [22]:
with graph.as_default():
    x2 = tf.Variable(2)

In [23]:
x2.graph is graph

True

In [24]:
x2.graph is tf.get_default_graph()

False

# Lifecycle of a Node Value

In [26]:
w = tf.constant(3)

In [27]:
x = w + 2
y = x + 5
z = x * 3

In [28]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [29]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


# Linear Regression with Tensorflow

In [30]:
import numpy as np 
from sklearn.datasets import fetch_california_housing

In [31]:
housing = fetch_california_housing()
m, n = housing.data.shape

In [32]:
m

20640

In [33]:
n

8

In [36]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [37]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [39]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)

In [40]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [41]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [42]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

# Manually Computing the Gradient

In [43]:
n_epochs = 1000
learning_rate = 0.01

In [47]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_hosuing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_hosuing_data]

In [60]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1, 1), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 7.0145574
Epoch 100 MSE = 0.8152255
Epoch 200 MSE = 0.655724
Epoch 300 MSE = 0.6202245
Epoch 400 MSE = 0.596789
Epoch 500 MSE = 0.57946247
Epoch 600 MSE = 0.56649446
Epoch 700 MSE = 0.556741
Epoch 800 MSE = 0.5493725
Epoch 900 MSE = 0.54377925


In [61]:
best_theta

array([[ 2.0685523 ],
       [ 0.88686085],
       [ 0.16125475],
       [-0.3142515 ],
       [ 0.3198532 ],
       [ 0.01021527],
       [-0.04414203],
       [-0.53629   ],
       [-0.51073265]], dtype=float32)

In [59]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1, 1), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0] ###
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 12.868064
Epoch 100 MSE = 0.8615995
Epoch 200 MSE = 0.62710655
Epoch 300 MSE = 0.5959593
Epoch 400 MSE = 0.5763387
Epoch 500 MSE = 0.56220335
Epoch 600 MSE = 0.5519505
Epoch 700 MSE = 0.54450446
Epoch 800 MSE = 0.539091
Epoch 900 MSE = 0.53515095


In [62]:
best_theta

array([[ 2.0685523 ],
       [ 0.88686085],
       [ 0.16125475],
       [-0.3142515 ],
       [ 0.3198532 ],
       [ 0.01021527],
       [-0.04414203],
       [-0.53629   ],
       [-0.51073265]], dtype=float32)

# Using an Optimizer

In [63]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

In [64]:
training_op = optimizer.minimize(mse)

# Feeding data to the Training Algorithm

In [84]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [85]:
A = tf.placeholder(tf.float32, shape=(None, 3))

In [86]:
B = A + 5

In [87]:
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

In [88]:
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [89]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
n_epochs = 10

In [90]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [91]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indicies = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indicies]
    y_batch = housing.target.reshape(-1, 1)[indicies]
    return X_batch, y_batch

In [92]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_theta = theta.eval()

In [93]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

# Saving and Restoring Models

In [102]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1, 1, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
            print("Epoch", epoch, "MSE =", mse.eval())  
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

best_theta

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.5555719
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436362
Epoch 600 MSE = 0.5396294
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340678
Epoch 900 MSE = 0.5321474


array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.6375277 ]], dtype=float32)

In [103]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restore = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [104]:
best_theta_restore

array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.6375277 ]], dtype=float32)

# TensorBoard

In [105]:
from datetime import datetime

In [106]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [111]:
reset_graph()


n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     # not shown

In [112]:
file_writer.close()

In [113]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

In [115]:
# from tensorflow_graph_in_jupyter import show_graph
# show_graph(tf.get_default_graph())

# Name Scopes

In [123]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

sess = tf.Session()
file_writer = tf.summary.FileWriter(logdir, sess.graph)
init = tf.global_variables_initializer()
sess.run(init)

In [117]:
error.op.name

'loss/sub'

In [118]:
mse.op.name

'loss/mse'

In [119]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [120]:
file_writer.close()

# Modularity

In [124]:
reset_graph()

In [126]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0., name="bias1")
b2 = tf.Variable(0., name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z2, 0., name="relu2")

output = tf.add(relu1, relu2, name="output")

In [131]:
print(X.get_shape()[1])
int(X.get_shape()[1])

3


3

In [140]:
reset_graph()
def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0., name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0, name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [141]:
file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

# Sharing Variables

In [144]:
reset_graph()
def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0., name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, i) for i in range(5)]
output = tf.add_n(relus, name="output")

In [145]:
file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())

In [147]:
reset_graph()
def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0., name="threshold")
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0., name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, relu.threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [148]:
file_writer = tf.summary.FileWriter("logs/relu3", tf.get_default_graph())

In [159]:
reset_graph()
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(), 
                                initializer=tf.constant_initializer(0.))

In [161]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

In [162]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

In [164]:
reset_graph()
def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0., name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                               initializer=tf.constant_initializer(0.))
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [165]:
file_writer = tf.summary.FileWriter("logs/relu4", tf.get_default_graph())

In [166]:
reset_graph()
def relu(X):
    threshold = tf.get_variable("threshold", shape=(), 
                               initializer=tf.constant_initializer(0.))
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0., name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [167]:
file_writer = tf.summary.FileWriter("logs/relu5", tf.get_default_graph())

# Exercises

In [168]:
import tensorflow as tf

In [170]:
x = tf.Variable(tf.random_uniform(shape=(), minval=0., maxval=1.))
x_new_val = tf.placeholder(shape=(), dtype=tf.float32)
x_assign = tf.assign(x, x_new_val)

In [171]:
with tf.Session():
    x.initializer.run()
    print(x.eval())
    x_assign.eval(feed_dict={x_new_val: 5.})
    print(x.eval())

0.97265816
5.0
